In [1]:
import time
from binance.client import Client
import pyodbc
import pandas as pd
import threading

client = Client()

# List of coins to fetch data for
coins = ["sxp", "chess", "blz", "joe", "perl", "ach", "gmt", "xrp", "akro", "zil", "cfx", "adx", "chz", "bel", "alpaca", "elf", "epx", "pros", "t", "dar", "agix", "mob", "id", "trx", "key", "tru", "amb", "magic", "lina", "lever"]

def get_db_connection():
    connection = pyodbc.connect(
        driver='{SQL Server}',
        server='A2NWPLSK14SQL-v06.shr.prod.iad2.secureserver.net',
        database='db_ran',
        uid='dbahsantrade',
        pwd='Pak@1947'
    )
    return connection

# Create tables for each coin
def create_tables():
    connection = get_db_connection()
    cursor = connection.cursor()
    for coin in coins:
        cursor.execute(f"""
            IF NOT EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'{coin}usdt') AND type in (N'U'))
            BEGIN
                CREATE TABLE {coin}usdt (
                    id INT IDENTITY(1,1) PRIMARY KEY,
                    timestamp DATETIME DEFAULT GETDATE(),
                    price DECIMAL(18, 8),
                    volume DECIMAL(18, 8)
                )
            END
        """)
    connection.commit()
    connection.close()

def fetch_and_store_recent_trades(coin):
    last_trade_timestamp = None
    while True:
        try:
            connection = get_db_connection()
            cursor = connection.cursor()
            trades = client.get_recent_trades(symbol=f'{coin.upper()}USDT')
            for trade in trades:
                trade_timestamp = pd.to_datetime(trade['time'], unit='ms')
                if last_trade_timestamp is None or trade_timestamp > last_trade_timestamp:
                    cursor.execute(f"INSERT INTO {coin}usdt (timestamp, price, volume) VALUES (?, ?, ?)",
                                   (trade_timestamp, trade['price'], trade['qty']))
                    last_trade_timestamp = trade_timestamp
            connection.commit()
        except Exception as e:
            print(f'Error fetching and storing trades for {coin}: {e}')
            connection.rollback()
            # Wait for 30 seconds before retrying
            time.sleep(30)
            continue
        finally:
            connection.close()
        time.sleep(5)
# Function to delete data older than 7 Days
def delete_old_data(coin):
    while True:
        try:
            connection = get_db_connection()
            cursor = connection.cursor()
            
            # Deleting in batches of 1000 rows to avoid large transactions
            rows_deleted = 1  # Initialize with dummy value to enter the loop
            while rows_deleted > 0:
                cursor.execute(f"""
                    DELETE TOP (1000)
                    FROM {coin}usdt
                    WHERE timestamp < DATEADD(DAY, -7, GETDATE())
                """)
                rows_deleted = cursor.rowcount
                connection.commit()
        except Exception as e:
            print(f'Error deleting old data for {coin}: {e}')
            if connection:
                connection.rollback()
        finally:
            if connection:
                connection.close()
        
        # Sleep for a certain period before running the deletion again. This example uses 6 hours.
        time.sleep(21600)  # Sleep for 6 hours



# Start the threads
def start_threads():
    create_tables()
    for coin in coins:
        threading.Thread(target=fetch_and_store_recent_trades, args=(coin,)).start()
        threading.Thread(target=delete_old_data, args=(coin,)).start()

start_threads()

while True:
    time.sleep(2)


Error fetching and storing trades for adx: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The transaction log for database 'db_ran' is full due to 'ACTIVE_TRANSACTION'. (9002) (SQLExecDirectW)")
Error fetching and storing trades for chz: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The transaction log for database 'db_ran' is full due to 'ACTIVE_TRANSACTION'. (9002) (SQLExecDirectW)")
Error fetching and storing trades for pros: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The transaction log for database 'db_ran' is full due to 'ACTIVE_TRANSACTION'. (9002) (SQLExecDirectW)")
Error fetching and storing trades for gmt: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The transaction log for database 'db_ran' is full due to 'ACTIVE_TRANSACTION'. (9002) (SQLExecDirectW)")
Error fetching and storing trades for tru: ('42000', "[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]The transaction log for database '

Exception in thread Thread-23:
Traceback (most recent call last):
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_10568\4046898997.py", line 51, in fetch_and_store_recent_trades
pyodbc.Error: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionRead (recv()). (65534) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_10568\4046898997.py", line 57, in fetch_and_store_recent_trades
pyodbc.OperationalError: ('08S01', '[08S01] [Microsoft][ODBC SQL Server Driver]Communication link failure (0) (SQLEndTr

Error fetching and storing trades for cfx: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionRead (recv()). (65534) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')
Error fetching and storing trades for joe: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionRead (recv()). (65534) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')


Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_10568\4046898997.py", line 51, in fetch_and_store_recent_trades
pyodbc.Error: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionRead (recv()). (65534) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_10568\4046898997.py", line 57, in fetch_and_store_recent_trades
pyodbc.OperationalError: ('08S01', '[08S01] [Microsoft][ODBC SQL Server Driver]Communication link failure (0) (SQLEndTra

Error fetching and storing trades for bel: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionWrite (send()). (10054) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')
Error fetching and storing trades for perl: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionWrite (send()). (10054) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')
Error fetching and storing trades for ach: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionRead (recv()). (65534) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')


Exception in thread Thread-13:
Traceback (most recent call last):
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_10568\4046898997.py", line 51, in fetch_and_store_recent_trades
pyodbc.Error: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionRead (recv()). (65534) (SQLExecDirectW); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\Ask78\anaconda3\envs\bapi\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Ask78\AppData\Local\Temp\ipykernel_10568\4046898997.py", line 57, in fetch_and_store_recent_trades
pyodbc.OperationalError: ('08S01', '[08S01] [Microsoft][ODBC SQL Server Driver]Communication link failure (0) (SQLEndTr